In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ýimport pickle

all_texts = pickle.load(open('/content/drive/My Drive/psych/klamanie/dane/all_texts.pkl', 'rb'))
all_labels = pickle.load(open('/content/drive/My Drive/psych/klamanie/dane/all_labels.pkl', 'rb'))


### ------------------------------------ USE

In [ ]:
#!pip3 install tensorflow-gpu
!pip3 install --quiet tensorflow-hub
!pip3 install tensorflow_text #>=2.0.0

     |████████████████████████████████| 4.9 MB 4.2 MB/s 


In [ ]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/2")


In [ ]:

all_embedded_texts = {}
n=50

for text_type in ['p', 't']:

  all_embedded_texts[text_type] = []
  chunks=[all_texts[text_type][i:i + n] for i in range(0, len(all_texts[text_type]), n)]  
  for chunk in chunks:
    all_embedded_texts[text_type].extend( embed(chunk).numpy() )



In [ ]:
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import NuSVC
from xgboost import XGBClassifier
import pickle
from scipy.stats import wilcoxon


#for kernel_type in ['rbf', 'linear']:
kernel_type = 'rbf'

use3_results = {}
for text_type in ['p', 't']:

  print(f"{text_type} ")
  embeddings = np.array(all_embedded_texts[text_type])
  labels = np.array(all_labels[text_type])

  use3_results[text_type] = []
  true_labels = []
  predicted_labels = []

  foldnr=0
  kf = KFold(n_splits=10, shuffle=True, random_state=42)
  for train_index, test_index in kf.split(embeddings):

    X_train, X_test = embeddings[train_index], embeddings[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    clf = NuSVC(nu=0.5, kernel=kernel_type)

    model = clf.fit(X_train, y_train.ravel())
    predictions = model.predict(X_test)

    use3_results[text_type].append( {'y_pred':predictions , 'y_true':y_test } )

    predicted_labels.extend(predictions)
    true_labels.extend(y_test)

    foldnr += 1

  print(classification_report(true_labels, predicted_labels, digits=2 ))

  stat, p = wilcoxon(x=predicted_labels, y=true_labels)
  print('Statistics=%.3f, p=%.3f' % (stat, p))
  # interpret
  alpha = 0.05
  if p > alpha:
    print('Same distribution (fail to reject H0)')
  else:
    print('Different distribution (reject H0)')


pickle.dump(use3_results, open('/content/drive/My Drive/psych/klamanie/experiments/use3_results-svmrbf.pkl','wb'))

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import NuSVC
from xgboost import XGBClassifier
import numpy as np
import warnings

Y = np.array(labels)
X = embeddings

#for kernel_type in ['rbf', 'linear']:
kernel_type = 'rbf'
n = 0.05
while n < 1.0:
  all_predicted_y = []
  loo = KFold(n_splits=20)
  for train_index, test_index in loo.split(Y):
      X_train, X_test = X[train_index], X[test_index]
      Y_train, Y_test = Y[train_index], Y[test_index]

      clf = NuSVC(nu=n, kernel=kernel_type)

      model = clf.fit(X_train, Y_train.ravel())
      predictions = model.predict(X_test)
      all_predicted_y.extend( predictions )

  acc = accuracy_score(Y, all_predicted_y)
  std_error = np.std( np.abs(np.subtract(Y, all_predicted_y)) ) / np.sqrt( len(all_predicted_y) )
  print("nu: %1.3f accuracy: %1.4f std_error: %1.4f" % (n, acc, std_error))
  n += 0.05


nu: 0.050 accuracy: 0.7082 std_error: 0.0168
nu: 0.100 accuracy: 0.7055 std_error: 0.0169
nu: 0.150 accuracy: 0.7041 std_error: 0.0169
nu: 0.200 accuracy: 0.7041 std_error: 0.0169
nu: 0.250 accuracy: 0.6986 std_error: 0.0170
nu: 0.300 accuracy: 0.6986 std_error: 0.0170
nu: 0.350 accuracy: 0.7041 std_error: 0.0169
nu: 0.400 accuracy: 0.7014 std_error: 0.0169
nu: 0.450 accuracy: 0.7055 std_error: 0.0169
nu: 0.500 accuracy: 0.7151 std_error: 0.0167
nu: 0.550 accuracy: 0.7123 std_error: 0.0168
nu: 0.600 accuracy: 0.7110 std_error: 0.0168
nu: 0.650 accuracy: 0.7151 std_error: 0.0167
nu: 0.700 accuracy: 0.7096 std_error: 0.0168
nu: 0.750 accuracy: 0.7164 std_error: 0.0167
nu: 0.800 accuracy: 0.7192 std_error: 0.0166
nu: 0.850 accuracy: 0.7205 std_error: 0.0166
nu: 0.900 accuracy: 0.7041 std_error: 0.0169
nu: 0.950 accuracy: 0.6808 std_error: 0.0173


In [ ]:


from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import NuSVC
from xgboost import XGBClassifier
import numpy as np
import warnings

for text_type in ['p', 't']:

  print(f"{text_type} ")

  Y = np.array(all_labels[text_type])
  X = np.array(all_embedded_texts[text_type])

  #for kernel_type in ['rbf', 'linear']:
  kernel_type = 'rbf'
  n = 0.05
  while n < 1.0:
    all_predicted_y = []
    loo = KFold(n_splits=20)
    for train_index, test_index in loo.split(Y):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        clf = NuSVC(nu=n, kernel=kernel_type)

        model = clf.fit(X_train, Y_train.ravel())
        predictions = model.predict(X_test)
        all_predicted_y.extend( predictions )

    acc = accuracy_score(Y, all_predicted_y)
    std_error = np.std( np.abs(np.subtract(Y, all_predicted_y)) ) / np.sqrt( len(all_predicted_y) )
    print("nu: %1.3f accuracy: %1.4f std_error: %1.4f" % (n, acc, std_error))
    n += 0.05


p 
nu: 0.050 accuracy: 0.6697 std_error: 0.0171
nu: 0.100 accuracy: 0.6697 std_error: 0.0171
nu: 0.150 accuracy: 0.6711 std_error: 0.0171
nu: 0.200 accuracy: 0.6684 std_error: 0.0171
nu: 0.250 accuracy: 0.6671 std_error: 0.0171
nu: 0.300 accuracy: 0.6697 std_error: 0.0171
nu: 0.350 accuracy: 0.6764 std_error: 0.0170
nu: 0.400 accuracy: 0.6869 std_error: 0.0169
nu: 0.450 accuracy: 0.6909 std_error: 0.0168
nu: 0.500 accuracy: 0.6869 std_error: 0.0169
nu: 0.550 accuracy: 0.6856 std_error: 0.0169
nu: 0.600 accuracy: 0.6843 std_error: 0.0169
nu: 0.650 accuracy: 0.6737 std_error: 0.0170
nu: 0.700 accuracy: 0.6684 std_error: 0.0171
nu: 0.750 accuracy: 0.6539 std_error: 0.0173
nu: 0.800 accuracy: 0.6539 std_error: 0.0173
nu: 0.850 accuracy: 0.6367 std_error: 0.0175
nu: 0.900 accuracy: 0.6341 std_error: 0.0175
nu: 0.950 accuracy: 0.5945 std_error: 0.0178
t 
nu: 0.050 accuracy: 0.6521 std_error: 0.0176
nu: 0.100 accuracy: 0.6493 std_error: 0.0177
nu: 0.150 accuracy: 0.6493 std_error: 0.0177
nu: 

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold, GridSearchCV



---



In [ ]:
import pickle

use3_results = pickle.load(open('/content/drive/My Drive/psych/klamanie/experiments/use3_results-svmrbf.pkl','rb'))


In [ ]:
from sklearn.metrics import classification_report

# http://rasbt.github.io/mlxtend/user_guide/evaluate/cochrans_q/
from mlxtend.evaluate import cochrans_q

import numpy as np
all_results = pickle.load( open('/content/drive/My Drive/psych/klamanie/experiments/in_domain_results.pkl','rb'))

#model_results = { 'report':report, 'y_pred':y_pred, 'y_true':y_test , 'fold_nr':fold_nr, 'model':BERT_MODEL['name'], 'text_type':text_type }
#all_results[text_type].append(model_results)


for text_type in ['p', 't']:
  print(f"text type: {text_type}")  
  y_pred = {}
  y_true = {}

  for result_dict in all_results[text_type]:
    if result_dict['model'] not in y_pred:
      y_pred[ result_dict['model'] ] = []
      y_true[ result_dict['model'] ] = []
    y_pred[ result_dict['model'] ].extend( result_dict['y_pred'].astype(int) )
    y_true[ result_dict['model'] ].extend( result_dict['y_true'].astype(int) )

  #use3_results[text_type].append( {'y_pred':predictions , 'y_true':y_test } )
  y_pred['use3'] = []
  for result_dict in use3_results[text_type]:
    y_pred['use3'].extend( result_dict['y_pred'] )

  q, p_value = cochrans_q(np.array(y_true['bert-base-uncased']), 
                          np.array(y_pred['allegro/herbert-base-cased']), 
                          np.array(y_pred['use3']) )

  print('Q: %.3f' % q)
  print('p-value: %.3f' % p_value)


text type: p
Q: 0.067
p-value: 0.796
text type: t
Q: 0.097
p-value: 0.756


/usr/local/lib/python3.7/dist-packages/mlxtend/evaluate/cochrans_q.py:75: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  mod in y_model_predictions)).astype(int)




---

human baseline

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
# http://rasbt.github.io/mlxtend/user_guide/evaluate/cochrans_q/
from mlxtend.evaluate import cochrans_q
# https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
from scipy.stats import wilcoxon

print('-'*20)
print('pisemne')

tresh = 2

y_true = [] ; y_pred = []
df = pd.read_excel('/content/drive/My Drive/psych/klamanie/experiments/p_czy_k_Aleks.xlsx', sheet_name='pisemne', index_col=None, header=0)
df = df.fillna('')
for index, row in df.iterrows():
  if row['id_t'] is not '' or row['id_f'] is not '':

    #print(row['id_t'], row['wiar_prawda'], row['id_f'], row['wiar_kłamstwo'] )

    if row['wiar_prawda'] is not '':
      if int(row['wiar_prawda']) >= tresh:
        y_true.append(1) ; y_pred.append(1)
      else:
        y_true.append(1) ; y_pred.append(0)
        
    if row['wiar_kłamstwo'] is not '':
      if int(row['wiar_kłamstwo']) >= tresh:
        y_true.append(0) ; y_pred.append(0)
      else:
        y_true.append(0) ; y_pred.append(1)

  else:
    break

print(classification_report(y_true, y_pred, digits=2 ))  
stat, p = wilcoxon(y_true, y_pred)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distribution (fail to reject H0)')
else:
	print('Different distribution (reject H0)')


print('-'*20)
print('transkrypcje')

y_true = [] ; y_pred = []
df = pd.read_excel('/content/drive/My Drive/psych/klamanie/experiments/p_czy_k_Aleks.xlsx', sheet_name='transkrypcje', index_col=None, header=0)
df = df.fillna('')
for index, row in df.iterrows():
  if row['id_t'] is not '' or row['id_f'] is not '':
    #print(row['id_t'], row['wiar_prawda'], row['id_f'], row['wiar_kłamstwo'] )

    if row['wiar_prawda'] is not '':
      if int(row['wiar_prawda']) >= tresh:
        y_true.append(1) ; y_pred.append(1)
      else:
        y_true.append(1) ; y_pred.append(0)
        
    if row['wiar_kłamstwo'] is not '':
      if int(row['wiar_kłamstwo']) >= tresh:
        y_true.append(0) ; y_pred.append(0)
      else:
        y_true.append(0) ; y_pred.append(1)

  else:
    break

print(classification_report(y_true, y_pred, digits=2 ))  
stat, p = wilcoxon(y_true, y_pred)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distribution (fail to reject H0)')
else:
	print('Different distribution (reject H0)')

--------------------
pisemne
              precision    recall  f1-score   support

           0       0.57      0.53      0.55       381
           1       0.56      0.60      0.58       380

    accuracy                           0.56       761
   macro avg       0.56      0.56      0.56       761
weighted avg       0.56      0.56      0.56       761

Statistics=25551.000, p=0.139
Same distribution (fail to reject H0)
--------------------
transkrypcje
              precision    recall  f1-score   support

           0       0.62      0.41      0.50       367
           1       0.56      0.75      0.64       367

    accuracy                           0.58       734
   macro avg       0.59      0.58      0.57       734
weighted avg       0.59      0.58      0.57       734

Statistics=14014.000, p=0.000
Different distribution (reject H0)
